In [3]:

cd _scripts/

/Users/ben/prj/fab/_scripts


In [107]:
import datetime
venues.created_at.apply(lambda x: datetime.datetime(x))

TypeError: an integer is required (got type str)

In [111]:
t0 = venues.created_at.iloc[0]

In [124]:

items

datetime.datetime(2019, 12, 24, 21, 26, 50, tzinfo=<StaticTzInfo 'UTC\+00:00'>)

In [0]:

venues.created_at.apply(lambda x: datetime.datetime(x))

In [125]:


%load_ext autoreload
%autoreload


import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
from airtable import Airtable
import pandas as pd

fill_untappd.run()

import pandas as pd

conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: dateparser.parse(x)).max()


venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))


cans = pd.read_csv("../_data/cans.csv")
cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

venue_beers_annotated.to_csv("../_data/venue_beers.csv")

venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
venues_annotated.to_csv("../_data/venues.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 845977174}
{'count': 0, 'items': []}


TypeError: Input type must be str or unicode

In [103]:
venue_beers_annotated.columns

Index([            'venue_id',           'venue_name', 'primary_category_key',
           'primary_category',   'parent_category_id',           'categories',
                   'location',              'contact',           'foursquare',
                 'venue_icon',          'is_verified',            'user_name',
                        'bid',            'beer_name',           'beer_label',
                   'beer_abv',             'beer_ibu',     'beer_description',
                 'beer_style',              'has_had',          'beer_active',
                 'checkin_id',           'created_at',         'rating_score',
                            0,              'twitter',            'venue_url',
                            0,                  'lat',                  'lng',
              'venue_address',           'venue_city',        'venue_country',
                'venue_state',           'user_names',         'total_rating',
                 'n_checkins',             'can_code

In [86]:

venues_annotated

,venue_name,venue_slug,primary_category,contact,venue_id,lat,lng,venue_city,venue_address,venue_country,venue_city,venue_country,venue_state
0,Dorchester Brewing Company,dorchester-brewing-company,Nightlife Spot,"{'twitter': '@dorchesterbrew', 'venue_url': 'h...",4367121,42.3220,-71.0626,Boston,1250 Massachusetts Ave,United States,Boston,United States,MA
1,Flatbread Company at Sacco's Bowl Haven,flatbread-company-at-saccos-bowl-haven,Food,"{'twitter': '@flatbreadsville', 'venue_url': '...",12317,42.3957,-71.1237,Somerville,45 Day St,United States,Somerville,United States,MA
2,High Tor State Park,high-tor-state-park,Outdoors & Recreation,"{'twitter': '', 'venue_url': ''}",2006735,41.1911,-73.9891,New City,415 S Mountain Rd,United States,New City,United States,NY
3,Flynn's Irish Pub - Mansfield,flynns-irish-pub,Food,"{'twitter': '@FlynnsIrishPub', 'venue_url': 'h...",1265320,42.0297,-71.2180,Mansfield,219-223 N Main St,United States,Mansfield,United States,MA
4,Connecticut River,connecticut-river,Outdoors & Recreation,"{'twitter': '', 'venue_url': ''}",1010984,42.2633,-72.6063,South Hadley,,United States,South Hadley,United States,MA


In [68]:


%load_ext autoreload
%autoreload


import fill_untappd
import yaml, json
import os
from pprint import pprint
from airtable import Airtable
import pandas as pd

fill_untappd.run()

import pandas as pd
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins")

venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts],axis=1))


cans = pd.read_csv("../_data/cans.csv")
cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 845977174}
{'count': 0, 'items': []}


In [72]:
venue_beers_annotated.can_code

venue_slug                              beer_slug                            
dorchester-brewing-company              fab-fermentation-arts-squid-pro-quo      sqp
flatbread-company-at-saccos-bowl-haven  fab-fermentation-arts-balloon-factory     bf
high-tor-state-park                     fab-fermentation-arts-squid-pro-quo      sqp
flynns-irish-pub                        fab-fermentation-arts-balloon-factory     bf
connecticut-river                       fab-fermentation-arts-squid-pro-quo      sqp
Name: can_code, dtype: object

In [48]:
venue_beers_annotated.can_code

AttributeError: 'DataFrame' object has no attribute 'can_code'

,0
0,3568228
1,3589306


In [41]:

cans

,Unnamed: 0,code,ad_feature_type,layout,collection_id,untappd,abv,description,merch_types,beer_style,name,ad_feature_id,statement,ad_feature_html,head_graphic,artist_id,last_brewed,untappd_id
0,1,sqp,prints,can,156883058772,https://untappd.com/b/fab-squid-pro-quo/3568228,7.5,"<em class=""sqp-text fancy"">SQUID PRO QUO</em> ...","museum quality prints, hoodies and stickers",DOUBLE IPA,Squid Pro Quo,4405538553940,"""I imagined him floating over a psychedelic su...","<div class=""print-circle-image"" style=""back...",/assets/story/squid_head_only.png,andy,12/4/2019,3568228
1,2,bf,artwork,can,156883124308,https://untappd.com/b/fab-fermentation-arts-ba...,6.1,"<em class=""bf-text fancy"">BALLOON FACTORY</em>...","the original artwork, T-shirts and stickers",AMERICAN IPA,Balloon Factory,4405832253524,“This baby Trump balloon is a very orange Geor...,"<button type=""button"" class=""btn "" data-toggle...",/assets/story/just_peach.png,adam,12/3/2019,3589306


,venue_slug,beer_slug,venue_id,venue_name,primary_category_key,primary_category,parent_category_id,categories,location,contact,...,lat,lng,venue_address,venue_city,venue_country,venue_state,user_names,total_rating,n_checkins,code
0,dorchester-brewing-company,fab-fermentation-arts-squid-pro-quo,4367121.0,Dorchester Brewing Company,Nightlife Spot,Nightlife Spot,4d4b7105d754a06376d81259,"{'count': 2, 'items': [{'category_key': 'brewe...","{'venue_address': '1250 Massachusetts Ave', 'v...","{'twitter': '@dorchesterbrew', 'venue_url': 'h...",...,42.3220,-71.0626,1250 Massachusetts Ave,Boston,United States,MA,"fab_ben,fab_ben,fab_ben",5.00,3.0,sqp
2,flatbread-company-at-saccos-bowl-haven,fab-fermentation-arts-balloon-factory,12317.0,Flatbread Company at Sacco's Bowl Haven,Food,Food,4d4b7105d754a06374d81259,"{'count': 1, 'items': [{'category_key': 'pizza...","{'venue_address': '45 Day St', 'venue_city': '...","{'twitter': '@flatbreadsville', 'venue_url': '...",...,42.3957,-71.1237,45 Day St,Somerville,United States,MA,dgerzofsky,0.00,1.0,bf
3,high-tor-state-park,fab-fermentation-arts-squid-pro-quo,2006735.0,High Tor State Park,Outdoors & Recreation,Outdoors & Recreation,4d4b7105d754a06377d81259,"{'count': 3, 'items': [{'category_key': 'state...","{'venue_address': '415 S Mountain Rd', 'venue_...","{'twitter': '', 'venue_url': ''}",...,41.1911,-73.9891,415 S Mountain Rd,New City,United States,NY,fab_ben,0.00,1.0,sqp
5,flynns-irish-pub,fab-fermentation-arts-balloon-factory,1265320.0,Flynn's Irish Pub - Mansfield,Food,Food,4d4b7105d754a06374d81259,"{'count': 1, 'items': [{'category_key': 'irish...","{'venue_address': '219-223 N Main St', 'venue_...","{'twitter': '@FlynnsIrishPub', 'venue_url': 'h...",...,42.0297,-71.2180,219-223 N Main St,Mansfield,United States,MA,hoppybeachbum,3.25,1.0,bf
6,connecticut-river,fab-fermentation-arts-squid-pro-quo,1010984.0,Connecticut River,Outdoors & Recreation,Outdoors & Recreation,4d4b7105d754a06377d81259,"{'count': 1, 'items': [{'category_key': 'river...","{'venue_address': '', 'venue_city': 'South Had...","{'twitter': '', 'venue_url': ''}",...,42.2633,-72.6063,,South Hadley,United States,MA,CatamountGold,4.00,1.0,sqp


In [24]:
venues_annotated.venue_id.astype(int)

0    4367121
2      12317
3    2006735
5    1265320
6    1010984
Name: venue_id, dtype: int64

In [9]:
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins limit 100;", conn)

,cid,timestamp,beer,venue,json
0,838537272,1575735296,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 838537272, ""created_at"": ""Sat, ..."
1,840159586,1576099768,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 840159586, ""created_at"": ""Wed, ..."
2,843595254,1576871652,Squid Pro Quo,,"{""checkin_id"": 843595254, ""created_at"": ""Fri, ..."
3,843598886,1576871947,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 843598886, ""created_at"": ""Fri, ..."
4,844777755,1577033160,Balloon Factory,Flatbread Company at Sacco's Bowl Haven,"{""checkin_id"": 844777755, ""created_at"": ""Sun, ..."
5,844837932,1577039748,Squid Pro Quo,High Tor State Park,"{""checkin_id"": 844837932, ""created_at"": ""Sun, ..."
6,845367612,1577132123,Balloon Factory,,"{""checkin_id"": 845367612, ""created_at"": ""Mon, ..."
7,845370113,1577132354,Squid Pro Quo,,"{""checkin_id"": 845370113, ""created_at"": ""Mon, ..."
8,845401472,1577135193,Squid Pro Quo,,"{""checkin_id"": 845401472, ""created_at"": ""Mon, ..."
9,845483235,1577142903,Squid Pro Quo,,"{""checkin_id"": 845483235, ""created_at"": ""Mon, ..."
